1. Configuration de la base de données

In [ ]:
import sqlite3

def init_db():
    conn = sqlite3.connect('expressions.db')
    cursor = conn.cursor()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS expressions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        expression TEXT NOT NULL
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS links (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        expr1_id INTEGER,
        expr2_id INTEGER,
        reason TEXT,
        FOREIGN KEY (expr1_id) REFERENCES expressions (id),
        FOREIGN KEY (expr2_id) REFERENCES expressions (id)
    )
    ''')
    
    conn.commit()
    conn.close()

init_db()


2. Interface graphique avec Tkinter

In [ ]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

# Initialisation de la base de données
def init_db():
    conn = sqlite3.connect('expressions.db')
    cursor = conn.cursor()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS expressions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        expression TEXT NOT NULL
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS links (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        expr1_id INTEGER,
        expr2_id INTEGER,
        reason TEXT,
        FOREIGN KEY (expr1_id) REFERENCES expressions (id),
        FOREIGN KEY (expr2_id) REFERENCES expressions (id)
    )
    ''')
    
    conn.commit()
    conn.close()

init_db()

# Ajout d'une nouvelle expression
def add_expression():
    expression = entry_expression.get()
    if expression:
        conn = sqlite3.connect('expressions.db')
        cursor = conn.cursor()
        cursor.execute('INSERT INTO expressions (expression) VALUES (?)', (expression,))
        conn.commit()
        conn.close()
        entry_expression.delete(0, tk.END)
        update_expression_list()
    else:
        messagebox.showwarning("Input Error", "Expression cannot be empty")

# Mise à jour de la liste des expressions
def update_expression_list():
    listbox_expressions.delete(0, tk.END)
    conn = sqlite3.connect('expressions.db')
    cursor = conn.cursor()
    cursor.execute('SELECT expression FROM expressions')
    expressions = cursor.fetchall()
    for expr in expressions:
        listbox_expressions.insert(tk.END, expr[0])
    conn.close()

# Mise à jour de la liste des liens
def update_links_list():
    listbox_links.delete(0, tk.END)
    conn = sqlite3.connect('expressions.db')
    cursor = conn.cursor()
    cursor.execute('''
    SELECT e1.expression, e2.expression, l.reason 
    FROM links l
    JOIN expressions e1 ON l.expr1_id = e1.id
    JOIN expressions e2 ON l.expr2_id = e2.id
    ''')
    links = cursor.fetchall()
    for link in links:
        listbox_links.insert(tk.END, f"{link[0]} --({link[2]})--> {link[1]}")
    conn.close()

# Suppression d'un lien
def delete_link():
    selected_link = listbox_links.get(tk.ACTIVE)
    if selected_link:
        parts = selected_link.split(' --(')
        expr1 = parts[0]
        reason = parts[1].split(')--> ')[0]
        expr2 = parts[1].split(')--> ')[1]
        
        conn = sqlite3.connect('expressions.db')
        cursor = conn.cursor()
        
        cursor.execute('SELECT id FROM expressions WHERE expression=?', (expr1,))
        expr1_id = cursor.fetchone()
        
        cursor.execute('SELECT id FROM expressions WHERE expression=?', (expr2,))
        expr2_id = cursor.fetchone()
        
        if expr1_id and expr2_id:
            cursor.execute('DELETE FROM links WHERE expr1_id=? AND expr2_id=? AND reason=?', (expr1_id[0], expr2_id[0], reason))
            conn.commit()
            conn.close()
            update_links_list()
        else:
            messagebox.showwarning("Delete Error", "Unable to find the selected link in the database")

# Suppression d'une expression
def delete_expression():
    selected_expr = listbox_expressions.get(tk.ACTIVE)
    if selected_expr:
        conn = sqlite3.connect('expressions.db')
        cursor = conn.cursor()
        
        cursor.execute('SELECT id FROM expressions WHERE expression=?', (selected_expr,))
        expr_id = cursor.fetchone()
        
        if expr_id:
            cursor.execute('DELETE FROM links WHERE expr1_id=? OR expr2_id=?', (expr_id[0], expr_id[0]))
            cursor.execute('DELETE FROM expressions WHERE id=?', (expr_id[0],))
            conn.commit()
            conn.close()
            update_expression_list()
            update_links_list()
        else:
            messagebox.showwarning("Delete Error", "Unable to find the selected expression in the database")

# Modification d'une expression
def edit_expression():
    selected_expr = listbox_expressions.get(tk.ACTIVE)
    new_expr = entry_expression.get()
    if selected_expr and new_expr:
        conn = sqlite3.connect('expressions.db')
        cursor = conn.cursor()
        cursor.execute('UPDATE expressions SET expression=? WHERE expression=?', (new_expr, selected_expr))
        conn.commit()
        conn.close()
        entry_expression.delete(0, tk.END)
        update_expression_list()
        update_links_list()
    else:
        messagebox.showwarning("Edit Error", "Both the selected and new expression cannot be empty")

# Variables globales pour la sélection des expressions
selected_expr1 = None
selected_expr2 = None

# Sélection de la première expression pour le lien
def select_expr1(event):
    global selected_expr1
    selected_expr1 = listbox_expressions.get(tk.ACTIVE)
    label_selected_expr1.config(text=f"Expression 1: {selected_expr1}")

# Sélection de la seconde expression pour le lien
def select_expr2(event):
    global selected_expr2
    selected_expr2 = listbox_expressions.get(tk.ACTIVE)
    label_selected_expr2.config(text=f"Expression 2: {selected_expr2}")

# Ajout d'un lien
def add_link():
    global selected_expr1, selected_expr2
    reason = entry_reason.get()
    
    if selected_expr1 and selected_expr2 and reason:
        conn = sqlite3.connect('expressions.db')
        cursor = conn.cursor()
        
        cursor.execute('SELECT id FROM expressions WHERE expression=?', (selected_expr1,))
        expr1_id = cursor.fetchone()
        
        cursor.execute('SELECT id FROM expressions WHERE expression=?', (selected_expr2,))
        expr2_id = cursor.fetchone()
        
        if expr1_id and expr2_id:
            cursor.execute('INSERT INTO links (expr1_id, expr2_id, reason) VALUES (?, ?, ?)', (expr1_id[0], expr2_id[0], reason))
            conn.commit()
            conn.close()
            entry_reason.delete(0, tk.END)
            update_links_list()
        else:
            messagebox.showwarning("Input Error", "Both expressions must exist in the database")
    else:
        messagebox.showwarning("Input Error", "Both expressions and reason are required")

# Visualisation de la carte mentale
def visualize_map():
    import networkx as nx
    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

    def show_graph():
        top = tk.Toplevel(root)
        top.title("Carte Mentale")

        conn = sqlite3.connect('expressions.db')
        cursor = conn.cursor()

        cursor.execute('SELECT id, expression FROM expressions')
        expressions = cursor.fetchall()

        cursor.execute('SELECT expr1_id, expr2_id, reason FROM links')
        links = cursor.fetchall()

        G = nx.DiGraph()

        for expr in expressions:
            G.add_node(expr[1])

        for link in links:
            expr1 = next(expr[1] for expr in expressions if expr[0] == link[0])
            expr2 = next(expr[1] for expr in expressions if expr[0] == link[1])
            G.add_edge(expr1, expr2, reason=link[2])

        pos = nx.spring_layout(G)
        fig, ax = plt.subplots(figsize=(10, 8))
        nx.draw(G, pos, with_labels=True, node_size=2000, node_color="skyblue", font_size=10, font_weight="bold", arrows=True, ax=ax)
        edge_labels = nx.get_edge_attributes(G, 'reason')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', ax=ax)

        canvas = FigureCanvasTkAgg(fig, master=top)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=1)

        conn.close()

    show_graph()

# Création de la fenêtre principale
root = tk.Tk()
root.title("Gestion des Expressions et Liens")

# Formulaire pour ajouter/modifier une expression
frame_expression = tk.Frame(root)
frame_expression.pack(pady=10)

label_expression = tk.Label(frame_expression, text="Nouvelle Expression:")
label_expression.pack(side=tk.LEFT)

entry_expression = tk.Entry(frame_expression)
entry_expression.pack(side=tk.LEFT)

button_add_expression = tk.Button(frame_expression, text="Ajouter", command=add_expression)
button_add_expression.pack(side=tk.LEFT)

button_edit_expression = tk.Button(frame_expression, text="Modifier", command=edit_expression)
button_edit_expression.pack(side=tk.LEFT)

# Liste des expressions
listbox_expressions = tk.Listbox(root, height=10, width=50)
listbox_expressions.pack(pady=10)
listbox_expressions.bind("<<ListboxSelect>>", select_expr1)
listbox_expressions.bind("<Double-1>", select_expr2)
update_expression_list()

button_delete_expression = tk.Button(root, text="Supprimer Expression", command=delete_expression)
button_delete_expression.pack(pady=10)

# Indicateurs pour les expressions sélectionnées pour les liens
label_selected_expr1 = tk.Label(root, text="Expression 1: Aucune")
label_selected_expr1.pack()
label_selected_expr2 = tk.Label(root, text="Expression 2: Aucune")
label_selected_expr2.pack()

# Formulaire pour ajouter/modifier un lien
frame_link = tk.Frame(root)
frame_link.pack(pady=10)

label_reason = tk.Label(frame_link, text="Raison du Lien:")
label_reason.pack(side=tk.LEFT)

entry_reason = tk.Entry(frame_link)
entry_reason.pack(side=tk.LEFT)

button_add_link = tk.Button(frame_link, text="Ajouter Lien", command=add_link)
button_add_link.pack(side=tk.LEFT)

# Liste des liens
listbox_links = tk.Listbox(root, height=10, width=100)
listbox_links.pack(pady=10)
update_links_list()

button_delete_link = tk.Button(root, text="Supprimer Lien", command=delete_link)
button_delete_link.pack(pady=10)

# Bouton pour visualiser la carte mentale
button_visualize = tk.Button(root, text="Visualiser Carte Mentale", command=visualize_map)
button_visualize.pack(pady=20)

root.mainloop()
